In [18]:
import pickle
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from p2_helper import Model

## Sensitive Cases

In [24]:
# effect on building stock 
a = Model()
a.pollution_lim = True
a.sensitivity = True
a.run_sim(time_steps=101)
a.plot_sim()

In [80]:
# effect on building stock 
b = Model()
b.pollution_lim = True
b.sensitivity = True
b.constants["lambda"] = a.constants["lambda"]*1.5
b.run_sim(time_steps=101)
b.plot_sim()

In [49]:
c = Model()
c.pollution_lim = True
c.sensitivity = True
c.constants["delb+"] = a.constants["delb+"]*1.5
c.run_sim(time_steps=101)
c.plot_sim()

In [74]:
c = Model()
c.pollution_lim = True
c.sensitivity = True
c.constants["delb-"] = a.constants["delb-"]*0.5
c.run_sim(time_steps=101)
c.plot_sim()

In [75]:
c = Model()
c.pollution_lim = True
c.sensitivity = True
c.constants["delb-"] = a.constants["delb-"]*1.5
c.run_sim(time_steps=101)
c.plot_sim()

## Analyze Sensitivity Data

In [13]:
s_data = pickle.load(open("sensitivity.p", "rb"))

In [14]:
key_names  = ["pol(t0)", "build(t0)", "coal(t0)", "solar(t0)",
'dels+',
 'delb+',
 'dels-',
 'delc-',
 'delb-',
 "pol_intensity", "pol_threshold", "pol_decay_param", 
 "urban_growth(t0)", "build_decom(t0)", "coal_extract(t0)", "solar_extract(t0)", "solar_inflow(t0)"  ]
keys = list(s_data.keys())
print(keys)

['p0', 'b0', 'c0', 's0', 'dels+', 'delb+', 'dels-', 'delc-', 'delb-', 'pi0', 'ap', 'lambda', 'u0', 'd0', 'ec0', 'es0', 'r0']


In [29]:
def get_high_low(time_step, concern, stock_or_flux, keys=keys, key_names=key_names):
    """
    time_step: 10, 25, 50, 100
    concern: coal, pollution, buildings, solar
    stock_or_flux: stock, flux
     """
    
    # extract the relevant data from sensitivity data 
    _dict = {}
    _dict["low"] = []
    _dict["original"] = []
    _dict["high"] = []
    for k in keys:
        _dict["low"].append(s_data[k][0.5][time_step][stock_or_flux][concern] )
        _dict["original"].append(s_data[k][1][time_step][stock_or_flux][concern])
        _dict["high"].append(s_data[k][1.5][time_step][stock_or_flux][concern])

    
    # only look at differences 
    result = {}
    # result["low"] = [np.abs(i - j) for i, j in zip(_dict["original"], _dict["low"])]

    # result["high"] = [np.abs(i - j) for i, j in zip(_dict["high"], _dict["original"])]

    result["low"] = [i - j for i, j in zip(_dict["original"], _dict["low"])]

    result["high"] = [i - j for i, j in zip(_dict["high"], _dict["original"])]

    # result["low"]  = [-1*i for i in result["low"]]

    return result


In [83]:
def plot_value_at_time(concerns, stock_or_flux, keys=keys, key_names=key_names, time=100):
    # time_steps = [10, 25, 50, 100]
    keys = key_names

    fig = make_subplots(rows=1, cols=4, shared_yaxes=True, subplot_titles=[c.title() for c in concerns])

    for ix, c in enumerate(concerns):
        show_legend_bool = True if ix == 0 else False
        result = get_high_low(time, c, stock_or_flux)
        fig.add_trace(go.Bar(
            y = key_names,
            x = result["low"],
            name = "-50%",
            orientation="h",
            marker_color = "rgb(158, 202, 225)",
            showlegend = show_legend_bool, 
        ), row = 1, col=ix+1)

        fig.add_trace(go.Bar(
            y = key_names,
            x = result["high"],
            name = "+50%",
            orientation="h",
            marker_color = "rgba(255,215,0,0.6)",
            showlegend = show_legend_bool, 
        ), row = 1, col=ix+1)

    fig.update_layout(title=f"Year {time} {stock_or_flux.title()} Sensitivity", barmode="overlay")

    return fig

In [88]:
concerns = ["buildings", "coal", "solar", "pollution"]
s_or_f = ["stock",  "flux"]
fig = plot_value_at_time(concerns, "stock")
fig.write_image(f"sens_figures/update_230517/yr_100_stock.png")
fig.show()

In [89]:
concerns = ["buildings", "coal", "solar", "pollution"]
fig = plot_value_at_time(concerns, "flux")
fig.write_image(f"sens_figures/update_230517/yr_100_flux.png")
fig.show()

In [71]:
def plot_value_over_time(concern, stock_or_flux, keys=keys, key_names=key_names):
    time_steps = [10, 25, 50, 100]
    keys = key_names

    fig = make_subplots(rows=1, cols=4, shared_yaxes=True, subplot_titles=[f"Year {t}" for t in time_steps])

    for ix, t in enumerate(time_steps):
        show_legend_bool = True if ix == 0 else False
        result = get_high_low(t, concern, stock_or_flux)
        fig.add_trace(go.Bar(
            y = key_names,
            x = result["low"],
            name = "-50%",
            orientation="h",
            marker_color = "rgb(158, 202, 225)",
            showlegend = show_legend_bool, 
        ), row = 1, col=ix+1)

        fig.add_trace(go.Bar(
            y = key_names,
            x = result["high"],
            name = "+50%",
            orientation="h",
            marker_color = "rgba(255,215,0,0.6)",
            showlegend = show_legend_bool, 
        ), row = 1, col=ix+1)

    fig.update_layout(title=f"{concern.title()} {stock_or_flux.title()}", barmode="overlay")

    return fig

In [73]:
concerns = ["buildings", "coal", "solar", "pollution"]
s_or_f = ["stock",  "flux"] # "flux"

for s in s_or_f:
    for c in concerns:
        t = 100
        result = get_high_low(t, c, s)
        fig = plot_value_over_time(c, s)
        print(c, result)
        fig.show()
        fig.write_image(f"sens_figures/update_230516/{c}_{s}.png", width=1200, height=700)
        # print(c,s)
    


buildings {'low': [0.0, -91.29722253960165, -10047.906943748541, 0.0, 0.0, 304.2839181405667, 0.0, -986.87836321977, -987.0711128108418, -64946.34500501119, 1.1368683772161603e-13, -21062.115648266816, 0.0, 0.0, -281.657671872524, 0.0, 0.0], 'high': [0.0, 529.1341566714242, 178.2346632998482, 0.0, 0.0, 1365.6131216365407, 0.0, 84.6614650674278, 338.70605631115575, -208.60489918730582, 2804.781811623518, -363.1045216390644, 0.0, 0.0, -2.2737367544323206e-13, 0.0, 0.0]}


coal {'low': [0.0, -1135.9113040160335, 1119.5970715420965, 0.0, 0.0, -13880.402928457903, 0.0, -1135.9113040160335, 1119.5970715420965, 1119.5970715420965, -10321.082410896022, 1119.5970715420965, 0.0, 0.0, -770.4326261047374, 0.0, 0.0], 'high': [0.0, -113.24988100889993, 11812.980188629317, 0.0, 0.0, -1119.5970715420965, 0.0, -113.24988100892938, 3351.557445549618, 8907.654802743033, -1119.5970715420965, 11747.337693587373, 0.0, 0.0, -459.8865834046318, 0.0, 0.0]}


solar {'low': [0.0, 0.0, 0.0, 0.0, 0.0, -105000.0, 0.0, 0.0, 0.0, 0.0, -108559.3205175618, 0.0, 0.0, 0.0, 0.0, -18696.47342487816, 0.0], 'high': [0.0, 99458.53086531791, 0.0, 0.0, 0.0, 0.0, 0.0, 96883.92938164978, 96133.01962459218, 109972.7481257149, 0.0, 107133.06523487056, 0.0, 0.0, 74567.56210142511, 0.0, 17541.360947042976]}


pollution {'low': [2.7983113341178978e-05, -282.4356343008253, 2564.861119304266, 0.0, 0.0, 1607.610576975268, 0.0, -282.4356343008253, 1823.7473582607045, 2565.687071566148, 1251.6433537081223, 2568.7460731853384, 0.0, 0.0, -147.04174526403767, 0.0, 0.0], 'high': [2.7983113341178978e-05, -443.07395085233657, -348.0838990359748, 0.0, 0.0, -1825.6113730554262, 0.0, -443.0739508523334, -388.81025708930565, -377.0211644643241, -2424.991478694651, 430.25556966458726, 0.0, 0.0, -265.9480188797352, 0.0, 0.0]}


buildings {'low': [0.0, 4.564861126980077, -582.8237390014838, 0.0, 0.0, -40.21419590702834, 0.0, 49.343918160988494, -174.56582322838398, -3327.7456420646163, -7.105427357601002e-15, -1133.5341742273974, 0.0, 0.0, 14.0828835936262, 0.0, 0.0], 'high': [0.0, 106.88509964762788, 89.34012497904908, 0.0, 0.0, 257.2038998847391, 0.0, 84.66146506742807, 68.7889452683214, 69.99814685469138, 220.66748239523258, 18.155226081953224, 0.0, 0.0, 80.42839181405667, 0.0, 0.0]}


coal {'low': [0.0, 68.25944450792025, -160.85678362811328, 0.0, 0.0, -60.85678362811328, 0.0, 68.25944450792025, -160.85678362811328, -160.85678362811328, -5.684341886080802e-14, -160.85678362811328, 0.0, 0.0, 6.331534374504713, 0.0, 0.0], 'high': [0.0, -55.82683133428489, -35.64693265996959, 0.0, 0.0, 160.85678362811328, 0.0, -55.8268313342852, -67.74121126223116, 41.720979837461186, 160.85678362811328, 72.62090432781298, 0.0, 0.0, -49.99999999999966, 0.0, 0.0]}


solar {'low': [0.0, -126.48421197399614, -691.042326042013, 68.12893459367274, -117.75207791680509, -1691.042326042013, -15.214195907028738, -61.926097905978736, -691.042326042013, -691.042326042013, -1751.8991096701263, -691.042326042013, 0.0, 0.0, 8.280995568831713, -249.99999999999977, 68.12893459367274], 'high': [0.0, 1551.7129414092633, 5.073903101774704, 0.0, -117.7520779168052, 691.042326042013, -83.34313050070148, 1540.0054776639563, 1454.4014063878826, 1710.178129832666, 691.042326042013, 1679.2782053423134, 0.0, 0.0, 1224.3452639827372, -318.12893459367274, 37.445110802827]}


pollution {'low': [-3.787102571095602e-06, -64.12650391245148, -76.6317716541144, 0.0, 0.0, -78.16913806935199, 0.0, -64.12650391245148, 23.667069107267352, -76.7435521374161, -130.00177441179613, -77.15753716990848, 0.0, 0.0, 9.113465774182316, 0.0, 0.0], 'high': [-3.787102571095602e-06, 115.79782089631374, 41.797055704107265, 0.0, 0.0, -23.41480213706646, 0.0, 115.79782089631397, 117.95474950278037, 109.00913010169268, 57.702474226008874, 52.558103371840375, 0.0, 0.0, 92.72460461422429, 0.0, 0.0]}


In [8]:
fig = plot_value_over_time("buildings", "flux")
fig